# Predictive Analytics – Simulation Study


In [ ]:
pip install simpy

In [ ]:
# Loading Libraries
import simpy
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Generating random set of parameters
def generate_random_parameters():
    arrival_rate = random.uniform(0.5, 5)
    service_rate = random.uniform(0.5, 6)
    sim_time = random.randint(50, 200)
    return arrival_rate, service_rate, sim_time

def customer(env, server, service_rate, wait_times):
    arrival_time = env.now
    with server.request() as request:
        yield request
        wait_times.append(env.now - arrival_time)
        service_time = np.random.exponential(1 / service_rate)
        yield env.timeout(service_time)

def run_simulation(arrival_rate, service_rate, sim_time):
    env = simpy.Environment()
    server = simpy.Resource(env, capacity = 1)
    wait_times = []
    def arrivals():
        while True:
            yield env.timeout(np.random.exponential(1 / arrival_rate))
            env.process(customer(env, server, service_rate, wait_times))
    env.process(arrivals())
    env.run(until = sim_time)
    return np.mean(wait_times) if wait_times else 0

simulation_data = []
num_simulations = 1000

for i in range(num_simulations):
    arrival_rate, service_rate, sim_time = generate_random_parameters()
    avg_wait_time = run_simulation(arrival_rate, service_rate, sim_time)
    simulation_data.append([
        arrival_rate,
        service_rate,
        sim_time,
        avg_wait_time
    ])

df = pd.DataFrame(simulation_data, columns = ["arrival_rate", "service_rate", "simulation_time", "avg_wait_time"])
print("Simulation Data\n", df)

In [ ]:
# Machine Learning Model Comparison
X = df[["arrival_rate", "service_rate", "simulation_time"]]
y = df["avg_wait_time"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 42)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(n_estimators = 100),
    "Support Vector Regression": SVR(),
    "KNN Regressor": KNeighborsRegressor()
}
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    r2 = r2_score(y_test, predictions)
    results.append({
        "Model": name,
        "RMSE": rmse,
        "R2 score": r2
    })
results_df = pd.DataFrame(results).sort_values("RMSE")
print("Model Comparison Table\n")
print(results_df)
best_model = results_df.iloc[0]
print("Best Performing Model\n")
print(best_model)

In [ ]:
# Graphical Display of Dataset
df_display = df.head(15).round(3)
plt.figure(figsize = (12, 6))
plt.axis('off')

table = plt.table(
    cellText = df_display.values,
    colLabels = df_display.columns,
    loc = 'center',
    cellLoc = 'center'
)

table.scale(1, 1.3)
table.auto_set_font_size(False)
table.set_fontsize(9)

plt.title("Simulation Dataset (First 15 Samples)", pad = 20)
plt.tight_layout()

In [ ]:
# Graphical display of table
results_df_rounded = results_df.round(3)
plt.figure(figsize = (10, 4))
plt.axis('off')

table = plt.table(
    cellText = results_df_rounded.values,
    colLabels = results_df_rounded.columns,
    loc = 'center',
    cellLoc = 'center'
)

table.scale(1, 1.5)
table.auto_set_font_size(False)
table.set_fontsize(11)

plt.title("Machine Learning Model Comparison", pad = 20)
plt.tight_layout()